**Nesse notebook, vou testar com w2v lgb e bayesopt**

## *Import Libraries*

In [2]:
import gc
import re
import operator 
import random
import time
import warnings
warnings.filterwarnings('ignore')
import os
import unidecode

import numpy as np
import pandas as pd
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
#from imblearn.metrics import sensitivity_score
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier

from xgboost import XGBClassifier
from string import punctuation

from stop_words import get_stop_words


Como o arquivo de treino é muito grando, vamos ter que limitar a quantidade de linhas utilizadas, pois só tenho 4gb ram. Também será necessário utilizar o encoder UTF-8, pois os textos são em português e espanhol

In [3]:
start_time = time.time()
p = 0.05
train = pd.read_csv('train.csv', header = 0, encoding = 'utf-8', skiprows = lambda i: i>0 and random.random() >p)
test = pd.read_csv('test.csv', header = 0, encoding = 'utf-8')

train.drop(['language','label_quality'], axis = 'columns', inplace = True) 
test.drop(['language'], axis = 'columns', inplace = True) 

print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

elapsed_time = int(time.time() - start_time)
print('{:02d}:{:02d}:{:02d}'.format(elapsed_time // 3600, (elapsed_time % 3600 // 60), elapsed_time % 60))


Train shape :  (997704, 2)
Test shape :  (246955, 2)
00:00:12


In [4]:
test.head()

,id,title
0,0,Kit Maternidade Bolsa-mala Baby/bebe Vinho Men...
1,1,Trocador De Fraldas Fisher Price Feminino Rosa...
2,2,Motor Ventoinha - Fiat Idea / Palio 1.8 - A 04...
3,3,Amortecedor Mola Batente D Dir New Civic 14 - ...
4,4,Cadeirinha De Carro Bebê Princesa Princess 9 A...


In [5]:
categorias = train.filter(['category'], axis = 1)
num = test.filter(['id'], axis = 1)
num.head()

,id
0,0
1,1
2,2
3,3
4,4


In [6]:
# Como nas competições é ideal juntar os datasets, iremos fazer isso agora:
train.drop(['category'], axis = 'columns', inplace = True) 
test.drop(['id'], axis = 'columns', inplace = True) 

df = pd.concat([train, test], axis=0)
del(train, test)
gc.collect()

656

In [7]:
df.shape

(1244659, 1)

## Embeddings and Preprocessing Text

**Aqui iremos criar as embbedings, palavras chaves para o nosso modelo treinar. É uma espécie de dicionário que deve conter pelo menos 95% do texto.**

In [8]:
# Algumas bibliotecas que temos que importar para poder realizar essa parte
# esses que estão desmarcados já foram importados.
#import pandas as pd
from tqdm import tqdm
tqdm.pandas()
#import operator
#import re
#from gensim.models import KeyedVectors

Nesse próximo passo iremos criar nosso vocabulário de teste. Essa função irá percorrer todo o texto e contar quantas ocorrências de cada palavra temos.

In [9]:
def build_vocab(sentences, verbose =  True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

Vamos montar o dicionário. A função *pregress_apply* permite que acompanhemos o tempo decorrido sobre um rotina

In [10]:
sentences = df["title"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

100%|████████████████████████████████████████████████████████████████████| 1244659/1244659 [00:02<00:00, 504284.06it/s]


{'4': 30836, 'Microaspersor': 44, 'Irrigação': 183, 'Ultra': 3349, '7,20': 8}


In [11]:
#aqui é a hora de carregar o modelo já treinado de dados, no caso do FastText
#import fastText
#trained_words = 'crawl-300d-2M.vec'
embeddings_index = KeyedVectors.load_word2vec_format('cbow_s300.txt')

In [ ]:
#aqui é a hora de carregar o modelo já treinado de dados, no caso do FastText
#import fastText
#trained_words = 'crawl-300d-2M.vec'
#embeddings_index2 = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.txt')

Agora, com essa função, vamos checar a intersecção entre nosso vocabulário e as embeddings. Ela vai gerar uma lista oov(output  out of vocabulary).

In [12]:
def check_coverage(vocab,embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in tqdm(vocab):
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:

            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.3%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.3%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    sorted_x = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [13]:
oov = check_coverage(vocab,embeddings_index)
gc.collect()

100%|██████████████████████████████████████████████████████████████████████| 604717/604717 [00:01<00:00, 562953.92it/s]


Found embeddings for 0.212% of vocab
Found embeddings for  3.309% of all text


0

In [14]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

In [15]:
df["title"] = df["title"].progress_apply(lambda x: clean_text(x))
sentences = df["title"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

100%|████████████████████████████████████████████████████████████████████| 1244659/1244659 [00:02<00:00, 460563.07it/s]


In [16]:
oov = check_coverage(vocab,embeddings_index)

100%|██████████████████████████████████████████████████████████████████████| 444556/444556 [00:00<00:00, 557976.20it/s]


Found embeddings for 3.690% of vocab
Found embeddings for  1.618% of all text


In [17]:
df['title'] = df['title'].apply(lambda x: x.lower())
gc.collect()

0

In [18]:
# remove 1 character terms (length == 1)
df['title'] = df['title'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))

In [19]:
vocab = build_vocab(df['title'])
oov = check_coverage(vocab,embeddings_index)

100%|████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:00<?, ?it/s]


Found embeddings for 85.149% of vocab
Found embeddings for  81.192% of all text


In [20]:
replace_1 = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\*)|(\d+)|(\+)|(\%)|(\&)|(\/)|(\-)|(\' ')|(\'°')|(\'\x9d')|(\'´')|(\'¡')|(\'®')|(\'¿')|(\'¨')|(\'×')|(\'»')|(\'»')|(\'·')|(\'¦')|(\'«')|(\'±')|(\'§')|(\'¢')|(\'£')|(\'\xad')|(\'\x81')")
                       
def clean_reviews(reviews):
    reviews = [replace_1.sub("", line.lower()) for line in reviews]
    return reviews
                       
df['title'] = clean_reviews(df['title'])

In [21]:
vocab = build_vocab(df['title'])
oov = check_coverage(vocab, embeddings_index)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:00<?, ?it/s]


Found embeddings for 93.407% of vocab
Found embeddings for  85.257% of all text


In [22]:
sentences = df["title"].progress_apply(lambda x: x.split())
to_remove = ['à',' ','\xa0','\x9d','\x81','\x7f','\x8d','\x90','°','\x9d','´','¡','®','¿','¨','×','»','·','¦','«','±','§','¢','£','\xad','\x81']
sentences = [[word for word in sentence if not word in to_remove] for sentence in tqdm(sentences)]
vocab = build_vocab(sentences)

100%|████████████████████████████████████████████████████████████████████| 1244659/1244659 [00:02<00:00, 617651.58it/s]


In [23]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'pçs':'peças',
                'soporte':'suporte',
                'delantero':'dianteiro',
                'cargador':'carregador',
                'lampara':'lâmpada',
                'talle':'tamanho',
                'cuotas':'dívidas',
                'embrague':'embreagem',
                'plegable':'dobrável',
                'inoxidable':'inoxidável',
                'impecable':'impecável',
                'accesorios':'acessórios',
                'inflable':'inflável',
                'estuche':'kit',
                'griferia':'torneira',
                'heladera': 'refrigerador',
                'compresor': 'compressor',
                'cubre': 'cobre',
                'silicona': 'silicone',
                'impresora': 'impressora'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [24]:
df["title"] = df["title"].apply(lambda x: replace_typical_misspell(x))

In [23]:
oov = check_coverage(vocab, embeddings_index)
oov[:20]

100%|██████████████████████████████████████████████████████████████████████| 237630/237630 [00:00<00:00, 475304.42it/s]


Found embeddings for 39.357% of vocab
Found embeddings for  90.375% of all text


[('talle', 5494),
 ('soporte', 4787),
 ('cuotas', 4682),
 ('delantero', 3981),
 ('cargador', 3559),
 ('embrague', 3323),
 ('pçs', 3175),
 ('plegable', 2777),
 ('inoxidable', 2771),
 ('lampara', 2705),
 ('impecable', 2498),
 ('accesorios', 2452),
 ('estuche', 2214),
 ('inflable', 2123),
 ('griferia', 1987),
 ('heladera', 1770),
 ('compresor', 1695),
 ('cubre', 1633),
 ('silicona', 1628),
 ('impresora', 1510)]

Podemos fazer uma limpeza maior e também processar um embedding do dicionário em espanhol. Mas por enquanto vamos ver como estamos fazendo.

In [24]:
#Primeiro vamos fazer a divisão do treino e teste de volta
train = df.iloc[:997704,:]
test = df.iloc[997704:,:]

print(train.shape)
print(test.shape)

(997704, 1)
(246955, 1)


In [25]:
del(df)
gc.collect()

0

In [26]:
#Agora vamos juntar novamente no treino a coluna de categoria e o id para teste
treino = pd.concat([train, categorias], axis=1)
treino.head()

,title,category
0,microaspersor irrigação ultra metros,IRRIGATION_SPRINKLERS
1,relógio de mesa despertador digital amarelo,ALARM_CLOCKS
2,oferta especial farinha de mandioca puba belém...,FLOUR
3,cdi pietcard original,MOTORCYCLE_CDI_MODULES
4,zapatilla con protector de tensión sobre consu...,STABILIZERS_AND_UPS


In [27]:
teste = pd.concat([num, test], axis=1)
teste.head()

,id,title
0,0,kit maternidade bolsa mala baby bebe vinho men...
1,1,trocador de fraldas fisher price feminino rosa...
2,2,motor ventoinha fiat idea palio
3,3,amortecedor mola batente dir new civic
4,4,cadeirinha de carro bebê princesa princess kgs


In [28]:
del(train)
del(test)
gc.collect()

7

Temos que convertar as strings para float, para poder alimentar o modelo. Iremos utilizar primeiro o BoW

In [29]:
def print_stats(preds, target, labels, sep='-', sep_len=40, fig_size=(10,8)):
    print('Accuracy = %.3f' % metrics.accuracy_score(target, preds))
    print(sep*sep_len)
    print('Classification report:')
    print(metrics.classification_report(target, preds))
    print(sep*sep_len)
    print('Confusion matrix')
    cm=metrics.confusion_matrix(target, preds)
    cm = cm / np.sum(cm, axis=1)[:,None]
    sns.set(rc={'figure.figsize':fig_size})
    sns.heatmap(cm, 
        xticklabels=labels,
        yticklabels=labels,
           annot=True, cmap = 'YlGnBu')
    plt.pause(0.05)

In [30]:
bow = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
bow_train = bow.fit_transform(treino['title'])
bow_test = bow.transform(teste['title'])

In [31]:
clf = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)

In [33]:
# splitting data into train, test, and validation set
X_train, X_test, y_train, y_test = train_test_split(treino['title'], treino['category'], 
                                                    test_size=0.33, random_state=42, stratify=treino['category'])

In [38]:
vec_bow = bow

In [39]:
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        # returns the input as a string
        return X[self.key]

In [47]:
text = Pipeline([
                ('selector', TextSelector(key='title')),
                ('vectorizer', vec_bow)
                ])

In [48]:
feats = FeatureUnion([('title', text)
                      ])

In [49]:
pipe = Pipeline([('feats', feats),
                 ('clf',clf)
                 ])

In [50]:
pipe.named_steps['clf'].get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.6,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 42,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 2,
 'silent': None,
 'subsample': 0.7,
 'verbosity': 1}

In [51]:
# definition of parameter grid to scan through
param_grid = {
     'clf__n_estimators': [50,100,300]
#    'clf__colsample_bytree': [0.6,0.8,1]
#    'clf__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

In [58]:
# grid search cross validation instantiation
from sklearn.svm import SVR
grid_search = GridSearchCV(estimator = clf, param_grid = param_grid, 
                          cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

In [59]:
#hyperparameter fitting
grid_search.fit(X_train, y_train)

IndexError: tuple index out of range

In [ ]:
grid_search.cv_results_['mean_train_score']

In [ ]:
grid_search.cv_results_['mean_test_score']

In [ ]:
grid_search.best_params_

In [ ]:
clf_test = grid_search.best_estimator_

In [ ]:
preds = clf_test.predict(bow_test)

In [ ]:
print_stats(y_test, preds, clf_test.classes_)

In [ ]:
submission_df = pd.read_csv('sample_submission.csv', header = 0, usecols = ['id'])
submission_df['category'] = preds
submission_df.to_csv("submission_xgb.csv", index=False)

Se quiser saber se o computador roda tensorflow
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

import tensorflow as tf
print(tf.__version__)